In [ ]:
import pyodbc
import pandas as pd
cnxn_string = r'Driver={SQL Server};Server=omop.uci.edu;Database=cords;Trusted_Connection=yes;'

In [ ]:
# All Procedures
cnxn = pyodbc.connect(cnxn_string)
'''
procedures = pd.DataFrame(pd.read_sql_query("""
with covid_positive_patients as (
        select m.person_id, min(m.measurement_date) as covid_positive_test_date from cords.dbo.measurement m, cords.dbo.concept c
        where m.measurement_concept_id = c.concept_id
        and vocabulary_id='LOINC'
        and concept_code in ('94500-6', '94309-2', '94531-1', '94500-6', '94310-0', '94533-7', '94306-8') and m.value_as_concept_id=9191
        group by person_id
),
p as (
    SELECT  a.[person_id], covid_positive_test_date, [procedure_concept_id], [procedure_date], [procedure_type_concept_id], [visit_occurrence_id]
	from covid_positive_patients a
	inner join (
		 select person_id, [procedure_concept_id], [procedure_date], [procedure_type_concept_id], [visit_occurrence_id]
		 FROM [CORDS].[dbo].[procedure_occurrence] 
		 where procedure_concept_id!=0) b
	on a.person_id = b.person_id
), 
with_procedure_name as (
	select c.concept_name as name_of_procedure, c.domain_id, p.* from p 
	left join (select * from cords.dbo.concept ) as c
		on c.concept_id=p.procedure_concept_id
	where c.domain_id='Procedure'
),
with_procedure_type as (
	select c.concept_name as procedure_type, p.* from with_procedure_name p
	left join (select * from cords.dbo.concept ) as c
		on c.concept_id=p.procedure_type_concept_id
),
procedures_after_covid_positive as (
select * from with_procedure_type where procedure_date >=covid_positive_test_date
)/*,
group_by_procedure as (
	select person_id, min(procedure_date) as procedure_date, name_of_procedure, procedure_concept_id from procedures_after_covid_positive group by person_id, name_of_procedure, procedure_concept_id
)*/,
select person_id, procedure_concept_id, procedure_date from group_by_procedure
""",cnxn))'''

procedures = pd.DataFrame(pd.read_sql_query("""
with visits as (
	select distinct vo.*, m.measurement_date
	from measurement m
	join concept c
	on m.measurement_concept_id = c.concept_id
	and c.vocabulary_id='LOINC' and c.concept_code IN ('94309-2', '94531-1', '94500-6' , '94310-0' , '94306-8', '94533-7')
	join visit_occurrence vo
	on m.person_id=vo.person_id
	and (m.measurement_date between dateadd(d, -30, vo.visit_start_date) and dateadd(d,1, vo.visit_start_date)
	or m.measurement_date between vo.visit_start_date and vo.visit_end_date
	)
	and vo.visit_concept_id=9201
	and m.value_as_concept_id=9191
),
with_procedures as (
	SELECT a.[person_id], measurement_date, [procedure_occurrence_id], [procedure_concept_id], [procedure_date],  a.[visit_occurrence_id], visit_start_date, visit_end_date
	FROM [CORDS].[dbo].[procedure_occurrence] a
	inner join (
		select * from visits
	) b
	on ((a.[visit_occurrence_id] = b.visit_occurrence_id) or (procedure_date between measurement_date and visit_end_date)  or (procedure_date between visit_start_date and visit_end_date))
	and a.person_id=b.person_id
	where procedure_date <= visit_end_date
)

--select person_id, procedure_concept_id from with_procedures group by person_id, procedure_concept_id
select person_id, procedure_concept_id, measurement_date, procedure_date, visit_start_date, visit_end_date
from with_procedures
where procedure_date <= visit_end_date

""",cnxn))



cnxn.close()



In [ ]:
procedures

In [ ]:
ecmo_concept_ids = [46257510,1531630,1531631,1531632,46257543,46257466,46257511,46257682,46257467]
ecmo_concept_ids = map(str, ecmo_concept_ids)
ecmo = procedures
ecmo = ecmo[ecmo['procedure_concept_id'].isin(ecmo_concept_ids)][["person_id"]]
ecmo['ecmo'] = '1'
ecmo_group_by_people = ecmo.groupby('person_id')
ecmo_indicator = ecmo_group_by_people['ecmo'].max()
ecmo_indicator.index = ecmo_indicator.index.map(str)

ventilation_concept_ids = map(str,[2314001, 2314000, 2788038, 2314035,2788037,2787823,2788036,2788019,2788024,2788028,2788023])
ventilation = procedures
ventilation = ventilation[ventilation['procedure_concept_id'].isin(ventilation_concept_ids)][["person_id"]]
ventilation['ventilation'] = '1'
ventilation_group_by_people = ventilation.groupby('person_id')
ventilation_indicator = ventilation_group_by_people['ventilation'].max()
ventilation_indicator.index = ventilation_indicator.index.map(str)

oxygen_concept_ids = [2314048,2106469,2108261, 2314047, 2314049]
oxygen_concept_ids = map(str, oxygen_concept_ids)
oxygen = procedures
oxygen = oxygen[oxygen['procedure_concept_id'].isin(oxygen_concept_ids)][["person_id"]]
oxygen['oxygen'] = '1'
oxygen_group_by_people = oxygen.groupby('person_id')
oxygen_indicator = oxygen_group_by_people['oxygen'].max()
oxygen_indicator.index = oxygen_indicator.index.map(str)
# oxygen_indicator

In [ ]:
len(ecmo_indicator)

In [ ]:
len(ventilation_indicator)

In [ ]:
len(oxygen_indicator)

In [ ]:
cnxn = pyodbc.connect(cnxn_string)
dialysis = pd.DataFrame(pd.read_sql_query("""
with covid_positive_patients as (
        select m.person_id, min(m.measurement_date) as covid_positive_test_date from cords.dbo.measurement m, cords.dbo.concept c
        where m.measurement_concept_id = c.concept_id
        and vocabulary_id='LOINC'
       and concept_code in ('94500-6', '94309-2', '94531-1', '94500-6', '94310-0', '94533-7', '94306-8') and m.value_as_concept_id=9191
        group by person_id
),
p as (
    SELECT  a.[person_id], covid_positive_test_date, [procedure_concept_id], [procedure_date], [procedure_type_concept_id], [visit_occurrence_id]
	from covid_positive_patients a
	inner join (
		 select person_id, [procedure_concept_id], [procedure_date], [procedure_type_concept_id], [visit_occurrence_id]
		 FROM [CORDS].[dbo].[procedure_occurrence] 
		 where procedure_concept_id!=0) b
	on a.person_id = b.person_id
), 
with_procedure_name as (
	select c.concept_name as name_of_procedure, c.domain_id, p.* from p 
	left join (select * from cords.dbo.concept ) as c
		on c.concept_id=p.procedure_concept_id
	where c.domain_id='Procedure'
),
with_procedure_type as (
	select c.concept_name as procedure_type, p.* from with_procedure_name p
	left join (select * from cords.dbo.concept ) as c
		on c.concept_id=p.procedure_type_concept_id
),
procedures_in_last_yr as (
select * from with_procedure_type where procedure_date >= dateadd(year, -1, covid_positive_test_date) 
and procedure_date <= dateadd(day, 1, covid_positive_test_date)
),
group_by_procedure as (
	select person_id, max(procedure_date) as procedure_date, name_of_procedure, procedure_concept_id from procedures_in_last_yr group by person_id, name_of_procedure, procedure_concept_id
)
select distinct person_id from group_by_procedure
where procedure_concept_id in (2213572,2213573,2213576,2213577,2213601,2617342)
""", cnxn))


In [ ]:
#dialysis[['person_id', 'name_of_procedure', 'procedure_concept_id']] = dialysis[['person_id', 'name_of_procedure', 'procedure_concept_id']].astype(str)
dialysis[['person_id']] = dialysis[['person_id']].astype(str)

# dialysis = dialysis.loc[dialysis['ancestor_concept_id']=='4032243']
#dialysis = dialysis.loc[dialysis['procedure_concept_id'].isin([2213572,2213573,2213576,2213577,2213601,2617342,2785872])]
dialysis['dialysis'] = '1'
# dialysis
# dialysis as Series to concat
dialysis_s = pd.Series(dialysis['dialysis'].values, index=dialysis['person_id'], name='dialysis', dtype=str)
# dialysis_s

In [ ]:
dialysis_s

In [ ]:
# Inpatient visits
cnxn = pyodbc.connect(cnxn_string)

inpatient_visits = pd.DataFrame(pd.read_sql_query("""
select distinct vo.*--, m.measurement_date, c.concept_code
from measurement m
join concept c
on m.measurement_concept_id = c.concept_id
and c.vocabulary_id='LOINC' and c.concept_code IN ('94309-2', '94531-1', '94500-6' , '94310-0' , '94306-8', '94533-7')
join visit_occurrence vo
on m.person_id=vo.person_id
and (m.measurement_date between dateadd(d, -30, vo.visit_start_date) and dateadd(d,1, vo.visit_start_date)
or m.measurement_date between vo.visit_start_date and vo.visit_end_date
)
and vo.visit_concept_id=9201
and m.value_as_concept_id=9191



""",cnxn))



cnxn.close()



In [ ]:
inpatient_visits

In [ ]:
inpatient_visits_copy = inpatient_visits.copy()

In [ ]:
inpatient_visits_copy = inpatient_visits_copy[['person_id', 'visit_start_date', 'visit_end_date']]
inpatient_visits_copy

In [ ]:
# Indicators in conditions table
cnxn = pyodbc.connect(cnxn_string)
conditions_after_diagnosis = pd.DataFrame(pd.read_sql_query(""" 
with inpatient_visits as (
	select distinct vo.person_id, vo.visit_concept_id, vo.visit_start_date, vo.visit_end_date, m.measurement_date
	from measurement m
	join concept c
	on m.measurement_concept_id = c.concept_id
	and c.vocabulary_id='LOINC' and c.concept_code IN ('94309-2', '94531-1', '94500-6' , '94310-0' , '94306-8', '94533-7')
	join visit_occurrence vo
	on m.person_id=vo.person_id
	and (m.measurement_date between dateadd(d, -30, vo.visit_start_date) and dateadd(d,1, vo.visit_start_date)
	or m.measurement_date between vo.visit_start_date and vo.visit_end_date
	)
	and vo.visit_concept_id=9201
	and m.value_as_concept_id=9191
),
conditions as (
	select * FROM [CORDS].[dbo].[condition_occurrence] co
	join concept c
	on co.condition_source_concept_id=c.concept_id and year(co.condition_start_date)>=2020 and c.concept_code in ('V46.11', 'V46.2')
),
procedures_during_inpatient_visits as (
select v.*, c.concept_code, c.condition_start_date from inpatient_visits v
join conditions c on c.person_id = v.person_id and ((c.condition_start_date between v.visit_start_date and v.visit_end_date) or (c.condition_start_date between v.measurement_date and v.visit_end_date))
)

select * from procedures_during_inpatient_visits
""", cnxn))
cnxn.close()

In [ ]:
conditions_after_diagnosis

In [ ]:
conditions_copy = conditions_after_diagnosis.copy() 

In [ ]:
conditions_copy = conditions_after_diagnosis.copy() 

ventilator_from_conditions = conditions_copy[conditions_copy['concept_code']=='V46.2'][['person_id']]
oxygen_from_conditions = conditions_copy[conditions_copy['concept_code']=='V46.11'][['person_id']]
ventilator_from_conditions.drop_duplicates('person_id', keep='first', inplace=True)
oxygen_from_conditions.drop_duplicates('person_id', keep='first', inplace=True)

In [ ]:
oxygen_from_conditions

In [ ]:
ventilator_from_conditions['ventilation'] = '1'
oxygen_from_conditions['oxygen'] = '1'

ventilator_from_conditions = ventilator_from_conditions.set_index('person_id')['ventilation']
oxygen_from_conditions = oxygen_from_conditions.set_index('person_id')['oxygen']

ventilator_from_conditions.index = ventilator_from_conditions.index.map(str)
oxygen_from_conditions.index = oxygen_from_conditions.index.map(str)

In [ ]:
ventilator_from_conditions

In [ ]:
oxygen_from_conditions

In [ ]:
a = pd.concat([ventilation_indicator, ventilator_from_conditions])
combined_ventilation =  a.reset_index().drop_duplicates(subset='person_id', keep='first').set_index('person_id')['ventilation']
combined_ventilation.index = combined_ventilation.index.map(str)

In [ ]:
b = pd.concat([oxygen_indicator, oxygen_from_conditions])
combined_oxygen =  b.reset_index().drop_duplicates(subset='person_id', keep='first').set_index('person_id')['oxygen']
combined_oxygen.index = combined_oxygen.index.map(str)

In [ ]:
a.reset_index().person_id.duplicated().value_counts()

In [ ]:
combined_oxygen
# oxygen_indicator
# combined_ventilation

In [ ]:
dialysis_s

In [ ]:
indicators = pd.concat([ecmo_indicator, combined_ventilation, combined_oxygen, dialysis_s], axis=1, join="outer")
indicators.fillna("0", inplace=True)
indicators.reset_index(inplace=True)
indicators.columns = ['person_id', 'ecmo', 'ventilation', 'oxygen', 'dialysis'] 
indicators[['person_id', 'ecmo', 'ventilation', 'oxygen', 'dialysis']] = indicators[['person_id','ecmo', 'ventilation', 'oxygen', 'dialysis']].astype(str)

In [ ]:
indicators

In [ ]:
# no duplicates
assert indicators[indicators['person_id'].duplicated(keep=False)==True].size == 0

In [ ]:
indicators.to_csv("procedures.csv", sep='\t', index=False)

In [ ]:
indicators.oxygen.value_counts()

In [ ]:
indicators.ventilation.value_counts()

In [ ]:
indicators.ecmo.value_counts()

In [ ]:
indicators.dialysis.value_counts()